In [77]:
import requests
import json
import os
import pandas
import util

In [ ]:
TOKEN = ""
BEACON_QUERY_URL = "https://beacon-emod-chem.maris.nl/api/query"

In [ ]:
query_parameters = []

# TIME
query_parameters.append(
    util.coalesce_columns(["time_ISO8601", "yyyy-mm-ddThh:mm:ss.sss"], "COMMON_TIME")
)
query_parameters.append(util.column("time_ISO8601_qc", "COMMON_TIME_QC"))
query_parameters.append(util.column("yyyy-mm-ddThh:mm:ss.sss"))

# LONGITUDE
query_parameters.append(util.column("Longitude"))
query_parameters.append(util.column("Longitude.units"))
# COMMON LONGITUDE
query_parameters.append(util.column("Longitude", "COMMON_LONGITUDE"))
query_parameters.append(util.value("degree_east", "COMMON_LONGITUDE_UNITS"))
query_parameters.append(util.value("longitude", "COMMON_LONGITUDE_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
query_parameters.append(util.value("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))

# LATITUDE
query_parameters.append(util.column("Latitude"))
query_parameters.append(util.column("Latitude.units"))
# COMMON LATITUDE
query_parameters.append(util.column("Latitude", "COMMON_LATITUDE"))
query_parameters.append(util.value("degree_north", "COMMON_LATITUDE_UNITS"))
query_parameters.append(util.value("latitude", "COMMON_LATITUDE_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
query_parameters.append(util.value("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))

# DEPTH
query_parameters.append(util.column("Depth"))
query_parameters.append(util.column("Depth.units"))
query_parameters.append(util.column("Depth_qc"))

# COMMON DEPTH
query_parameters.append(util.column("Depth", "COMMON_ORIGIN_DEPTH"))
query_parameters.append(util.column("Depth_qc", "COMMON_ORIGIN_DEPTH_QC"))
query_parameters.append(util.column("Depth.units", "COMMON_ORIGIN_DEPTH_UNITS"))
query_parameters.append(util.value("", "COMMON_ORIGIN_DEPTH_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ADEPZZ01", "COMMON_ORIGIN_DEPTH_P01"))
query_parameters.append(util.value("SDN:P06::ULAA", "COMMON_ORIGIN_DEPTH_P06"))

query_parameters.append(util.column("Depth", "COMMON_DEPTH"))
query_parameters.append(util.column("Depth_qc", "COMMON_DEPTH_QC"))
query_parameters.append(util.value("m", "COMMON_DEPTH_UNITS"))
query_parameters.append(util.value("depth", "COMMON_DEPTH_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
query_parameters.append(util.value("SDN:P06::ULAA", "COMMON_DEPTH_P06"))

In [ ]:
# DATASET METADATA
query_parameters.append(util.value("", "COMMON_EDMERP_CODE"))
# EDMO Code
query_parameters.append(util.column("EDMO_code", "COMMON_EDMO_CODE"))

# CSR
query_parameters.append(util.column("CSR platform code", "COMMON_CSR"))

# COMMON_ODV TAG
query_parameters.append(
    util.function_call("concat", ["EDMO_code", "LOCAL_CDI_ID"], "COMMON_ODV_TAG")
)

# BEACON SOURCE REFERENCE
query_parameters.append(util.value("BEACON_EMODNET_CHEMISTRY", "SOURCE_BDI"))
query_parameters.append(
    util.function_call("concat", ["EDMO_code", "LOCAL_CDI_ID"], "SOURCE_BDI_DATASET_ID")
)

# Platform code
query_parameters.append(
    util.function_call("map_emodnet_chemistry_platform_l06", ["Platform type"], "COMMON_PLATFORM_L06")
)

# ORIGIN METADATA FORWARDING
metadata = [
    "Cruise",
    "Cruise name",
    "CSR platform code",
    "CSR platform name",
    "Measuring area type",
    "Platform type",
    "Instrument / gear type",
    "LOCAL_CDI_ID",
    "EDMO_code",
    "P35 Contributor Codes",
    "featureType"
]
for m in metadata:
    query_parameters.append(util.column(m))
    
query_parameters.append(util.column("featureType", "COMMON_FEATURE_TYPE"))

In [ ]:
# MAP CHLOROPHYLL
query_parameters.append(util.column("Water body chlorophyll-a"))
query_parameters.append(util.column("Water body chlorophyll-a_qc"))
query_parameters.append(util.column("Water body chlorophyll-a.units"))

query_parameters.append(
    util.column("Water body chlorophyll-a", "COMMON_ORIGIN_CHLOROPHYLL")
)
query_parameters.append(
    util.column("Water body chlorophyll-a_qc", "COMMON_ORIGIN_CHLOROPHYLL_QC")
)
query_parameters.append(
    util.value("Water body chlorophyll-a", "COMMON_ORIGIN_CHLOROPHYLL_STANDARD_NAME")
)
query_parameters.append(
    util.column("Water body chlorophyll-a.units", "COMMON_ORIGIN_CHLOROPHYLL_UNITS")
)
query_parameters.append(
    util.value("SDN:P01::CHLTVOLU", "COMMON_ORIGIN_CHLOROPHYLL_P01")
)
query_parameters.append(util.value("SDN:P06::UMMC", "COMMON_ORIGIN_CHLOROPHYLL_P06"))

# MAP COMMON CHLOROPHYLL
query_parameters.append(
    util.column("Water body chlorophyll-a", "COMMON_CHLOROPHYLL_PER_VOLUME")
)
query_parameters.append(
    util.column("Water body chlorophyll-a_qc", "COMMON_CHLOROPHYLL_PER_VOLUME_QC")
)
query_parameters.append(
    util.value(
        "mass_concentration_of_chlorophyll_in_sea_water",
        "COMMON_CHLOROPHYLL_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("mg m-3", "COMMON_CHLOROPHYLL_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::CHLTVOLU", "COMMON_CHLOROPHYLL_PER_VOLUME_P01")
)
query_parameters.append(
    util.value("SDN:P06::UMMC", "COMMON_CHLOROPHYLL_PER_VOLUME_P06")
)
# MAP COMMON CHLOROPHYLL INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_emodnet_chemistry_instrument_l05", ["Instrument / gear type"], "COMMON_CHLOROPHYLL_L05"
    )
)
query_parameters.append(util.value("", "COMMON_CHLOROPHYLL_L22"))
query_parameters.append(util.value("", "COMMON_CHLOROPHYLL_L33"))

In [ ]:
# MAP OXYGEN
query_parameters.append(util.column("Water body dissolved oxygen concentration"))
query_parameters.append(util.column("Water body dissolved oxygen concentration_qc"))
query_parameters.append(util.column("Water body dissolved oxygen concentration.units"))

query_parameters.append(
    util.column("Water body dissolved oxygen concentration", "COMMON_ORIGIN_OXYGEN")
)
query_parameters.append(
    util.column(
        "Water body dissolved oxygen concentration_qc", "COMMON_ORIGIN_OXYGEN_QC"
    )
)
query_parameters.append(
    util.value(
        "Water body dissolved oxygen concentration",
        "COMMON_ORIGIN_OXYGEN_STANDARD_NAME",
    )
)
query_parameters.append(
    util.column(
        "Water body dissolved oxygen concentration.units", "COMMON_ORIGIN_OXYGEN_UNITS"
    )
)
query_parameters.append(util.value("SDN:P01::DOXYZZXX", "COMMON_ORIGIN_OXYGEN_P01"))
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_ORIGIN_OXYGEN_P06"))

# MAP COMMON OXYGEN PER VOLUME
query_parameters.append(
    util.column("Water body dissolved oxygen concentration", "COMMON_OXYGEN_PER_VOLUME")
)
query_parameters.append(
    util.column(
        "Water body dissolved oxygen concentration_qc", "COMMON_OXYGEN_PER_VOLUME_QC"
    )
)
query_parameters.append(
    util.value(
        "mole_concentration_of_dissolved_molecular_oxygen_in_sea_water",
        "COMMON_OXYGEN_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol l-1", "COMMON_OXYGEN_PER_VOLUME_UNITS"))
query_parameters.append(util.value("SDN:P01::DOXYZZXX", "COMMON_OXYGEN_PER_VOLUME_P01"))
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_OXYGEN_PER_VOLUME_P06"))

# MAP COMMON OXYGEN PER MASS
query_parameters.append(
    util.function_call(
        "map_units",
        [
            util.value("SDN:P06::UPOX"),
            util.value("SDN:P06::KGUM"),
            "Water body dissolved oxygen concentration",
        ],
        "COMMON_OXYGEN_PER_MASS",
    )
)
query_parameters.append(
    util.column(
        "Water body dissolved oxygen concentration_qc", "COMMON_OXYGEN_PER_MASS_QC"
    )
)
query_parameters.append(
    util.value(
        "moles_of_oxygen_per_unit_mass_in_sea_water",
        "COMMON_OXYGEN_PER_MASS_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol kg-1", "COMMON_OXYGEN_PER_MASS_UNITS"))
query_parameters.append(util.value("SDN:P01::DOXMZZXX", "COMMON_OXYGEN_PER_MASS_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_OXYGEN_PER_MASS_P06"))


# MAP COMMON OXYGEN INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_emodnet_chemistry_instrument_l05", ["Instrument / gear type"], "COMMON_OXYGEN_L05"
    )
)
query_parameters.append(util.value("", "COMMON_OXYGEN_L22"))
query_parameters.append(util.value("", "COMMON_OXYGEN_L33"))

In [ ]:
# MAP NITRATE
query_parameters.append(util.column("Water body nitrate"))
query_parameters.append(util.column("Water body nitrate_qc"))
query_parameters.append(util.column("Water body nitrate.units"))

query_parameters.append(util.column("Water body nitrate", "COMMON_ORIGIN_NITRATE"))
query_parameters.append(
    util.column("Water body nitrate_qc", "COMMON_ORIGIN_NITRATE_QC")
)
query_parameters.append(
    util.value(
        "Water body nitrate",
        "COMMON_ORIGIN_NITRATE_STANDARD_NAME",
    )
)
query_parameters.append(
    util.column("Water body nitrate.units", "COMMON_ORIGIN_NITRATE_UNITS")
)
query_parameters.append(util.value("SDN:P01::NTRAZZXX", "COMMON_ORIGIN_NITRATE_P01"))
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_ORIGIN_NITRATE_P06"))

# MAP COMMON NITRATE PER VOLUME
query_parameters.append(util.column("Water body nitrate", "COMMON_NITRATE_PER_VOLUME"))
query_parameters.append(
    util.column("Water body nitrate_qc", "COMMON_NITRATE_PER_VOLUME_QC")
)
query_parameters.append(
    util.value(
        "mole_concentration_of_nitrate_in_sea_water",
        "COMMON_NITRATE_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol l-1", "COMMON_NITRATE_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::NTRAZZXX", "COMMON_NITRATE_PER_VOLUME_P01")
)
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_NITRATE_PER_VOLUME_P06"))

# MAP COMMON NITRATE PER MASS
query_parameters.append(
    util.function_call(
        "map_units",
        [
            util.value("SDN:P06::UPOX"),
            util.value("SDN:P06::KGUM"),
            "Water body nitrate",
        ],
        "COMMON_NITRATE_PER_MASS",
    )
)
query_parameters.append(
    util.column("Water body nitrate_qc", "COMMON_NITRATE_PER_MASS_QC")
)
query_parameters.append(
    util.value(
        "moles_of_nitrate_per_unit_mass_in_sea_water",
        "COMMON_NITRATE_PER_MASS_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol kg-1", "COMMON_NITRATE_PER_MASS_UNITS"))
query_parameters.append(util.value("SDN:P01::MDMAP005", "COMMON_NITRATE_PER_MASS_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_NITRATE_PER_MASS_P06"))


# MAP COMMON NITRATE INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_emodnet_chemistry_instrument_l05", ["Instrument / gear type"], "COMMON_NITRATE_L05"
    )
)
query_parameters.append(util.value("", "COMMON_NITRATE_L22"))
query_parameters.append(util.value("", "COMMON_NITRATE_L33"))

In [ ]:
# MAP NITRATE + NITRITE
query_parameters.append(util.column("Water body nitrate plus nitrite"))
query_parameters.append(util.column("Water body nitrate plus nitrite_qc"))
query_parameters.append(util.column("Water body nitrate plus nitrite.units"))

query_parameters.append(util.column("Water body nitrate plus nitrite", "COMMON_ORIGIN_NITRATE_NITRITE_NITRITE"))
query_parameters.append(
    util.column("Water body nitrate plus nitrite_qc", "COMMON_ORIGIN_NITRATE_NITRITE_QC")
)
query_parameters.append(
    util.value(
        "Water body nitrate plus nitrite",
        "COMMON_ORIGIN_NITRATE_NITRITE_STANDARD_NAME",
    )
)
query_parameters.append(
    util.column("Water body nitrate plus nitrite.units", "COMMON_ORIGIN_NITRATE_NITRITE_UNITS")
)
query_parameters.append(util.value("SDN:P01::NTRZZZXX", "COMMON_ORIGIN_NITRATE_NITRITE_P01"))
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_ORIGIN_NITRATE_NITRITE_P06"))

# MAP COMMON NITRATE PER VOLUME
query_parameters.append(util.column("Water body nitrate plus nitrite", "COMMON_NITRATE_NITRITE_PER_VOLUME"))
query_parameters.append(
    util.column("Water body nitrate plus nitrite_qc", "COMMON_NITRATE_NITRITE_PER_VOLUME_QC")
)
query_parameters.append(
    util.value(
        "mole_concentration_of_nitrate_and_nitrite_in_sea_water",
        "COMMON_NITRATE_NITRITE_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol l-1", "COMMON_NITRATE_NITRITE_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::NTRZZZXX", "COMMON_NITRATE_NITRITE_PER_VOLUME_P01")
)
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_NITRATE_NITRITE_PER_VOLUME_P06"))

# MAP COMMON NITRATE PER MASS
query_parameters.append(
    util.function_call(
        "map_units",
        [
            util.value("SDN:P06::UPOX"),
            util.value("SDN:P06::KGUM"),
            "Water body nitrate plus nitrite",
        ],
        "COMMON_NITRATE_NITRITE_PER_MASS",
    )
)
query_parameters.append(
    util.column("Water body nitrate plus nitrite_qc", "COMMON_NITRATE_NITRITE_PER_MASS_QC")
)
query_parameters.append(
    util.value(
        "moles_of_nitrate_and_nitrite_per_unit_mass_in_sea_water",
        "COMMON_NITRATE_NITRITE_PER_MASS_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol kg-1", "COMMON_NITRATE_NITRITE_PER_MASS_UNITS"))
query_parameters.append(util.value("SDN:P01::MDMAP006", "COMMON_NITRATE_NITRITE_PER_MASS_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_NITRATE_NITRITE_PER_MASS_P06"))


# MAP COMMON NITRATE INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_emodnet_chemistry_instrument_l05", ["Instrument / gear type"], "COMMON_NITRATE_NITRITE_L05"
    )
)
query_parameters.append(util.value("", "COMMON_NITRATE_NITRITE_L22"))
query_parameters.append(util.value("", "COMMON_NITRATE_NITRITE_L33"))

In [ ]:
# MAP AMMONIUM
query_parameters.append(util.column("Water body ammonium"))
query_parameters.append(util.column("Water body ammonium_qc"))
query_parameters.append(util.column("Water body ammonium.units"))

query_parameters.append(util.column("Water body ammonium", "COMMON_ORIGIN_AMMONIUM"))
query_parameters.append(
    util.column("Water body ammonium_qc", "COMMON_ORIGIN_AMMONIUM_QC")
)
query_parameters.append(
    util.value(
        "Water body ammonium",
        "COMMON_ORIGIN_AMMONIUM_STANDARD_NAME",
    )
)
query_parameters.append(
    util.column("Water body ammonium.units", "COMMON_ORIGIN_AMMONIUM_UNITS")
)
query_parameters.append(util.value("SDN:P01::AMONZZXX", "COMMON_ORIGIN_AMMONIUM_P01"))
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_ORIGIN_AMMONIUM_P06"))

# MAP COMMON AMMONIUM PER VOLUME
query_parameters.append(
    util.column("Water body ammonium", "COMMON_AMMONIUM_PER_VOLUME")
)
query_parameters.append(
    util.column("Water body ammonium_qc", "COMMON_AMMONIUM_PER_VOLUME_QC")
)
query_parameters.append(
    util.value(
        "mole_concentration_of_ammonium_in_sea_water",
        "COMMON_AMMONIUM_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol l-1", "COMMON_AMMONIUM_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::AMONZZXX", "COMMON_AMMONIUM_PER_VOLUME_P01")
)
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_AMMONIUM_PER_VOLUME_P06"))

# MAP COMMON AMMONIUM PER MASS
query_parameters.append(
    util.function_call(
        "map_units",
        [
            util.value("SDN:P06::UPOX"),
            util.value("SDN:P06::KGUM"),
            "Water body ammonium",
        ],
        "COMMON_AMMONIUM_PER_MASS",
    )
)
query_parameters.append(
    util.column("Water body ammonium_qc", "COMMON_AMMONIUM_PER_MASS_QC")
)
query_parameters.append(
    util.value(
        "moles_of_ammonium_per_unit_mass_in_sea_water",
        "COMMON_AMMONIUM_PER_MASS_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol kg-1", "COMMON_AMMONIUM_PER_MASS_UNITS"))
query_parameters.append(util.value("SDN:P01::MDMAP004", "COMMON_AMMONIUM_PER_MASS_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_AMMONIUM_PER_MASS_P06"))


# MAP COMMON AMMONIUM INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_emodnet_chemistry_instrument_l05", ["Instrument / gear type"], "COMMON_AMMONIUM_L05"
    )
)
query_parameters.append(util.value("", "COMMON_AMMONIUM_L22"))
query_parameters.append(util.value("", "COMMON_AMMONIUM_L33"))

In [ ]:
# MAP PHOSPHATE
query_parameters.append(util.column("Water body phosphate"))
query_parameters.append(util.column("Water body phosphate_qc"))
query_parameters.append(util.column("Water body phosphate.units"))

query_parameters.append(util.column("Water body phosphate", "COMMON_ORIGIN_PHOSPHATE"))
query_parameters.append(
    util.column("Water body phosphate_qc", "COMMON_ORIGIN_PHOSPHATE_QC")
)
query_parameters.append(
    util.value(
        "Water body phosphate",
        "COMMON_ORIGIN_PHOSPHATE_STANDARD_NAME",
    )
)
query_parameters.append(
    util.column("Water body phosphate.units", "COMMON_ORIGIN_PHOSPHATE_UNITS")
)
query_parameters.append(util.value("SDN:P01::PHOSZZXX", "COMMON_ORIGIN_PHOSPHATE_P01"))
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_ORIGIN_PHOSPHATE_P06"))

# MAP COMMON PHOSPHATE PER VOLUME
query_parameters.append(
    util.column("Water body phosphate", "COMMON_PHOSPHATE_PER_VOLUME")
)
query_parameters.append(
    util.column("Water body phosphate_qc", "COMMON_PHOSPHATE_PER_VOLUME_QC")
)
query_parameters.append(
    util.value(
        "mole_concentration_of_phosphate_in_sea_water",
        "COMMON_PHOSPHATE_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol l-1", "COMMON_PHOSPHATE_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::PHOSZZXX", "COMMON_PHOSPHATE_PER_VOLUME_P01")
)
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_PHOSPHATE_PER_VOLUME_P06"))

# MAP COMMON PHOSPHATE PER MASS
query_parameters.append(
    util.function_call(
        "map_units",
        [
            util.value("SDN:P06::UPOX"),
            util.value("SDN:P06::KGUM"),
            "Water body phosphate",
        ],
        "COMMON_PHOSPHATE_PER_MASS",
    )
)
query_parameters.append(
    util.column("Water body phosphate_qc", "COMMON_PHOSPHATE_PER_MASS_QC")
)
query_parameters.append(
    util.value(
        "moles_of_phosphate_per_unit_mass_in_sea_water",
        "COMMON_PHOSPHATE_PER_MASS_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol kg-1", "COMMON_PHOSPHATE_PER_MASS_UNITS"))
query_parameters.append(
    util.value("SDN:P01::MDMAP906", "COMMON_PHOSPHATE_PER_MASS_P01")
)
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_PHOSPHATE_PER_MASS_P06"))


# MAP COMMON PHOSPHATE INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_emodnet_chemistry_instrument_l05", ["Instrument / gear type"], "COMMON_PHOSPHATE_L05"
    )
)
query_parameters.append(util.value("", "COMMON_PHOSPHATE_L22"))
query_parameters.append(util.value("", "COMMON_PHOSPHATE_L33"))

In [ ]:
# MAP SILICATE
query_parameters.append(util.column("Water body silicate"))
query_parameters.append(util.column("Water body silicate_qc"))
query_parameters.append(util.column("Water body silicate.units"))

query_parameters.append(util.column("Water body silicate", "COMMON_ORIGIN_SILICATE"))
query_parameters.append(
    util.column("Water body silicate_qc", "COMMON_ORIGIN_SILICATE_QC")
)
query_parameters.append(
    util.value(
        "Water body silicate",
        "COMMON_ORIGIN_SILICATE_STANDARD_NAME",
    )
)
query_parameters.append(
    util.column("Water body silicate.units", "COMMON_ORIGIN_SILICATE_UNITS")
)
query_parameters.append(util.value("SDN:P01::SLCAZZXX", "COMMON_ORIGIN_SILICATE_P01"))
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_ORIGIN_SILICATE_P06"))

# MAP COMMON SILICATE PER VOLUME
query_parameters.append(
    util.column("Water body silicate", "COMMON_SILICATE_PER_VOLUME")
)
query_parameters.append(
    util.column("Water body silicate_qc", "COMMON_SILICATE_PER_VOLUME_QC")
)
query_parameters.append(
    util.value(
        "mole_concentration_of_silicate_in_sea_water",
        "COMMON_SILICATE_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol l-1", "COMMON_SILICATE_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::SLCAZZXX", "COMMON_SILICATE_PER_VOLUME_P01")
)
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_SILICATE_PER_VOLUME_P06"))

# MAP COMMON SILICATE PER MASS
query_parameters.append(
    util.function_call(
        "map_units",
        [
            util.value("SDN:P06::UPOX"),
            util.value("SDN:P06::KGUM"),
            "Water body silicate",
        ],
        "COMMON_SILICATE_PER_MASS",
    )
)
query_parameters.append(
    util.column("Water body silicate_qc", "COMMON_SILICATE_PER_MASS_QC")
)
query_parameters.append(
    util.value(
        "moles_of_silicate_per_unit_mass_in_sea_water",
        "COMMON_SILICATE_PER_MASS_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol kg-1", "COMMON_SILICATE_PER_MASS_UNITS"))
query_parameters.append(util.value("SDN:P01::MDMAP012", "COMMON_SILICATE_PER_MASS_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_SILICATE_PER_MASS_P06"))


# MAP COMMON SILICATE INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_emodnet_chemistry_instrument_l05", ["Instrument / gear type"], "COMMON_SILICATE_L05"
    )
)
query_parameters.append(util.value("", "COMMON_SILICATE_L22"))
query_parameters.append(util.value("", "COMMON_SILICATE_L33"))

In [ ]:
# MAP TEMPERATURE
query_parameters.append(util.column("ITS-90 water temperature"))
query_parameters.append(util.column("ITS-90 water temperature_qc"))
query_parameters.append(util.column("ITS-90 water temperature.units"))

query_parameters.append(
    util.column("ITS-90 water temperature", "COMMON_ORIGIN_TEMPERATURE")
)
query_parameters.append(
    util.column("ITS-90 water temperature_qc", "COMMON_ORIGIN_TEMPERATURE_QC")
)
query_parameters.append(
    util.value("ITS-90 water temperature", "COMMON_ORIGIN_TEMPERATURE_STANDARD_NAME")
)
query_parameters.append(
    util.column("ITS-90 water temperature.units", "COMMON_ORIGIN_TEMPERATURE_UNITS")
)
query_parameters.append(
    util.value("SDN:P01::TEMPPR01", "COMMON_ORIGIN_TEMPERATURE_P01")
)
query_parameters.append(util.value("SDN:P06::UPAA", "COMMON_ORIGIN_TEMPERATURE_P06"))

# MAP COMMON TEMPERATURE
query_parameters.append(util.column("ITS-90 water temperature", "COMMON_TEMPERATURE"))
query_parameters.append(
    util.column("ITS-90 water temperature_qc", "COMMON_TEMPERATURE_QC")
)
query_parameters.append(
    util.value(
        "sea_water_temperature",
        "COMMON_TEMPERATURE_STANDARD_NAME",
    )
)
query_parameters.append(util.value("degree_C", "COMMON_TEMPERATURE_UNITS"))
query_parameters.append(util.value("SDN:P01::TEMPPR01", "COMMON_TEMPERATURE_P01"))
query_parameters.append(util.value("SDN:P06::UPAA", "COMMON_TEMPERATURE_P06"))
# MAP COMMON TEMPERATURE INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_emodnet_chemistry_instrument_l05", ["Instrument / gear type"], "COMMON_TEMPERATURE_L05"
    )
)
query_parameters.append(util.value("", "COMMON_TEMPERATURE_L22"))
query_parameters.append(util.value("", "COMMON_TEMPERATURE_L33"))

In [ ]:
# MAP SALINITY
query_parameters.append(util.column("Water body salinity"))
query_parameters.append(util.column("Water body salinity_qc"))
query_parameters.append(util.column("Water body salinity.units"))

query_parameters.append(util.column("Water body salinity", "COMMON_ORIGIN_SALINITY"))
query_parameters.append(
    util.column("Water body salinity_qc", "COMMON_ORIGIN_SALINITY_QC")
)
query_parameters.append(
    util.value("Water body salinity", "COMMON_ORIGIN_SALINITY_STANDARD_NAME")
)
query_parameters.append(
    util.column("Water body salinity.units", "COMMON_ORIGIN_SALINITY_UNITS")
)
query_parameters.append(util.value("SDN:P01::PSLTZZ01", "COMMON_ORIGIN_SALINITY_P01"))
query_parameters.append(util.value("SDN:P06::UUUU", "COMMON_ORIGIN_SALINITY_P06"))

# MAP COMMON SALINITY
query_parameters.append(util.column("Water body salinity", "COMMON_SALINITY"))
query_parameters.append(util.column("Water body salinity_qc", "COMMON_SALINITY_QC"))
query_parameters.append(
    util.value(
        "sea_water_salinity",
        "COMMON_SALINITY_STANDARD_NAME",
    )
)
query_parameters.append(util.value("1e-3", "COMMON_SALINITY_UNITS"))
query_parameters.append(util.value("SDN:P01::PSLTZZ01", "COMMON_SALINITY_P01"))
query_parameters.append(util.value("SDN:P06::UUUU", "COMMON_SALINITY_P06"))
# MAP COMMON SALINITY INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_emodnet_chemistry_instrument_l05", ["Instrument / gear type"], "COMMON_SALINITY_L05"
    )
)
query_parameters.append(util.value("", "COMMON_SALINITY_L22"))
query_parameters.append(util.value("", "COMMON_SALINITY_L33"))

In [90]:
def build_query(start_time, end_time) -> dict:
    return {
        "select": query_parameters,
        "filters": [
            {
                "column": "yyyy-mm-ddThh:mm:ss.sss",
                "min": start_time,
                "max": end_time,
            }
        ],
        "output": {"format": "parquet"},
    }

In [91]:
os.makedirs("data", exist_ok=True)

for year in range(1950, 2024):
    start_time = f"{year}-01-01T00:00:00.000"
    end_time = f"{year}-12-31T23:59:59.999"
    print(f"Downloading {start_time} - {end_time}")
    # Download data
    with requests.post(
        BEACON_QUERY_URL,
        json=build_query(start_time, end_time),
        headers={"Authorization": f"Bearer {TOKEN}"},
        stream=True,
    ) as response:
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            print(response.text)
            exit(1)
        response.raise_for_status()
        with open(f"data/EMODnet_chemistry_{year}.parquet", "wb") as f:
            for chunk in response.iter_content(chunk_size=4096 * 128):
                if chunk:
                    f.write(chunk)

In [92]:
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile("data/EMODnet_chemistry_2000.parquet")

# Read each row group as a pandas DataFrame
print(parquet_file.num_row_groups)
table = parquet_file.read_row_group(0)
df = table.to_pandas()
df

1


,COMMON_TIME,COMMON_TIME_QC,yyyy-mm-ddThh:mm:ss.sss,Longitude,Longitude.units,COMMON_LONGITUDE,COMMON_LONGITUDE_UNITS,COMMON_LONGITUDE_STANDARD_NAME,COMMON_LONGITUDE_P01,COMMON_LONGITUDE_P06,...,COMMON_ORIGIN_SALINITY_P06,COMMON_SALINITY,COMMON_SALINITY_QC,COMMON_SALINITY_STANDARD_NAME,COMMON_SALINITY_UNITS,COMMON_SALINITY_P01,COMMON_SALINITY_P06,COMMON_SALINITY_L05,COMMON_SALINITY_L22,COMMON_SALINITY_L33
0,2000-04-12 23:10:00,None,2000-04-12 23:10:00,6.2142,degrees_east,6.2142,degrees_east,Longitude,SDN:P01::ALONZZ01,SDN:P06::DEGE,...,SDN:P06::UUUU,34.599998,1,Salinity of the water body,Dimensionless,SDN:P01::PSLTZZ01,SDN:P06::UUUU,SDN:L05::999,,
1,2000-04-12 23:10:00,None,2000-04-12 23:10:00,6.2142,degrees_east,6.2142,degrees_east,Longitude,SDN:P01::ALONZZ01,SDN:P06::DEGE,...,SDN:P06::UUUU,33.939999,1,Salinity of the water body,Dimensionless,SDN:P01::PSLTZZ01,SDN:P06::UUUU,SDN:L05::999,,
2,2000-04-12 23:10:00,None,2000-04-12 23:10:00,6.2142,degrees_east,6.2142,degrees_east,Longitude,SDN:P01::ALONZZ01,SDN:P06::DEGE,...,SDN:P06::UUUU,33.130001,1,Salinity of the water body,Dimensionless,SDN:P01::PSLTZZ01,SDN:P06::UUUU,SDN:L05::999,,
3,2000-04-12 23:10:00,None,2000-04-12 23:10:00,6.2142,degrees_east,6.2142,degrees_east,Longitude,SDN:P01::ALONZZ01,SDN:P06::DEGE,...,SDN:P06::UUUU,34.320000,1,Salinity of the water body,Dimensionless,SDN:P01::PSLTZZ01,SDN:P06::UUUU,SDN:L05::999,,
4,2000-04-12 23:10:00,None,2000-04-12 23:10:00,6.2142,degrees_east,6.2142,degrees_east,Longitude,SDN:P01::ALONZZ01,SDN:P06::DEGE,...,SDN:P06::UUUU,34.000000,1,Salinity of the water body,Dimensionless,SDN:P01::PSLTZZ01,SDN:P06::UUUU,SDN:L05::999,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938503,2000-03-28 08:14:00,None,2000-03-28 08:14:00,12.8638,degrees_east,12.8638,degrees_east,Longitude,SDN:P01::ALONZZ01,SDN:P06::DEGE,...,SDN:P06::UUUU,8.694100,1,Salinity of the water body,Dimensionless,SDN:P01::PSLTZZ01,SDN:P06::UUUU,SDN:L05::30,,
938504,2000-03-28 08:14:00,None,2000-03-28 08:14:00,12.8638,degrees_east,12.8638,degrees_east,Longitude,SDN:P01::ALONZZ01,SDN:P06::DEGE,...,SDN:P06::UUUU,8.694200,1,Salinity of the water body,Dimensionless,SDN:P01::PSLTZZ01,SDN:P06::UUUU,SDN:L05::30,,
938505,2000-03-28 08:14:00,None,2000-03-28 08:14:00,12.8638,degrees_east,12.8638,degrees_east,Longitude,SDN:P01::ALONZZ01,SDN:P06::DEGE,...,SDN:P06::UUUU,8.703200,1,Salinity of the water body,Dimensionless,SDN:P01::PSLTZZ01,SDN:P06::UUUU,SDN:L05::30,,
938506,2000-03-28 08:14:00,None,2000-03-28 08:14:00,12.8638,degrees_east,12.8638,degrees_east,Longitude,SDN:P01::ALONZZ01,SDN:P06::DEGE,...,SDN:P06::UUUU,8.806500,1,Salinity of the water body,Dimensionless,SDN:P01::PSLTZZ01,SDN:P06::UUUU,SDN:L05::30,,
